In [1]:
###############################################################
# VectorStore, Retriever 준비
###############################################################
from dotenv import load_dotenv

load_dotenv()

True

In [2]:
from langchain_chroma import Chroma
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_openai import OpenAIEmbeddings

COLLECTION_NAME = "olympic_info"
PERSIST_DIRECTORY = "vector_store/chroma/olympic_info"

def get_vectorstore():
    embedding_model = OpenAIEmbeddings(
        model="text-embedding-3-large",
    )

    vector_store = Chroma(
        embedding_function=embedding_model,
        collection_name=COLLECTION_NAME,
        persist_directory=PERSIST_DIRECTORY
    )
    return vector_store


def get_retriever(k=10):
    vector_store = get_vectorstore()
    retriever = vector_store.as_retriever(search_kwargs={"k":k})
    return retriever

In [3]:
###########################################################################
# Document 추가
###########################################################################

from langchain_community.document_loaders import TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

vector_store = get_vectorstore()

path = 'data/olympic.txt'
loader = TextLoader(path, encoding="utf-8")
splitter = RecursiveCharacterTextSplitter(
    chunk_size=500, chunk_overlap=50
)

docs = loader.load_and_split(splitter)

ids = vector_store.add_documents(documents=docs)
len(ids)

61

In [4]:
retriever = get_retriever(k=3)
retriever.invoke("IOC에 대해 설명해줘.")

[Document(id='29e55886-576f-4292-a1c3-5489789fe9ac', metadata={'source': 'data/olympic.txt'}, page_content='국제 올림픽 위원회\n올림픽 활동이란 많은 수의 국가, 국제 경기 연맹과 협회 • 미디어 파트너를 맺기 • 선수, 직원, 심판, 모든 사람과 기관이 올림픽 헌장을 지키는 것을 말한다. 국제올림픽위원회(IOC)는 모든 올림픽 활동을 통솔하는 단체로서, 올림픽 개최 도시 선정, 계획 감독, 종목 변경, 스폰서 및 방송권 계약 체결 등의 권리가 있다. 올림픽 활동은 크게 세 가지로 구성된다.\n- 국제경기연맹(IF)은 국제적인 규모의 경기를 관리, 감독하는 기구이다. 예를 들어서 국제 축구 연맹(FIFA)는 축구를 주관하며, 국제 배구 연맹(FIVB)은 배구를 주관하는 기구이다. 올림픽에는 현재 35개의 국제경기연맹이 있고 각 종목을 대표한다. (이 중에는 올림픽 종목은 아니지만 IOC의 승인을 받은 연맹도 있다.)'),
 Document(id='5c551e09-4117-4759-a019-d5861bb37ea4', metadata={'source': 'data/olympic.txt'}, page_content='국제 올림픽 위원회\n올림픽 활동이란 많은 수의 국가, 국제 경기 연맹과 협회 • 미디어 파트너를 맺기 • 선수, 직원, 심판, 모든 사람과 기관이 올림픽 헌장을 지키는 것을 말한다. 국제올림픽위원회(IOC)는 모든 올림픽 활동을 통솔하는 단체로서, 올림픽 개최 도시 선정, 계획 감독, 종목 변경, 스폰서 및 방송권 계약 체결 등의 권리가 있다. 올림픽 활동은 크게 세 가지로 구성된다.\n- 국제경기연맹(IF)은 국제적인 규모의 경기를 관리, 감독하는 기구이다. 예를 들어서 국제 축구 연맹(FIFA)는 축구를 주관하며, 국제 배구 연맹(FIVB)은 배구를 주관하는 기구이다. 올림픽에는 현재 35개의 국제경기연맹이 있고 각 종목을 대표한다. (이 중에는 올림픽 종목은 아

# Rerank

## 개념

- RAG의 정확도는 관련 정보의 컨텍스트 내 존재 유무가 아니라 순서가 중요하다. 즉, 관련 정보가 컨텍스트 내 상위권에 위치하고 있을 때 좋은 답변을 얻을 수 있다는 뜻이다. 
- **Rerank**는 RAG 시스템에서 **초기 검색 단계에서 추출된 후보 문서들의 순위를 재조정**하는 기법이다. 
- 벡터 유사도 기반의 빠른 1차 검색 후, 보다 정밀한 모델(예: Cross-encoder, LLM 등)을 활용해 질문과 검색된 문서간의 의미론적 관련성을 평가하여, 실제로 답변 생성에 가장 **적합한 문서들이 상위**에 오도록 순서를 다시 매긴다. 이를 통해 LLM이 더 정확하고 관련성 높은 정보를 바탕으로 답변을 생성할 수 있게 도와준다.

## 방법

- **Cross-encoder 기반 Rerank**  
  - Cross Encoder를 이용해서 순위를 재 지정한다.
  - Cross-encoder
    - 질문과 문서를 같이 입력으로 받아 둘간의 유사도 점수를 예측하도록 학습한 모델.
    - 학습을 두 문장의 유사도록 예측하도록 학습하였기 때문에 단순 유사도 검사 보다 두 문장간의 의미적 관련성등을 이용해 유사도를 예측하기 때문에 더 정확한 결과를 보인다.
  - 1차적으로 검색한 문서와 질문간의 유사도를 **cross-encoder**로 다시 계산해서 문서의 순위를 재 조정한다.
  
  > - **Bi-encoder**
  >     - 질문 (query)와 문서(document)를 각각 독립적으로 인코딩한 후, 벡터 간 유사도 계산
  >     - Encoder 모델은 개별 문장을 입력받아 embedding vector를 출력한다. 질문과 문서를 각각 encoding한 뒤에 둘 간의 유사도를 계산한다.
  
  ![bi_crosss_encoder](figures/bi_cross_encoder.png)

  \[출처:https://aws.amazon.com/ko/blogs/tech/korean-reranker-rag/\]

- **LLM 기반 Rerank**  
  GPT-3, GPT-4 등 LLM을 활용해 각 문서가 질문에 얼마나 부합하는지 평가하여 순위를 매긴다. 성능이 뛰어나지만 비용이 높고 응답 속도가 느릴 수 있다.
## Rerank RAG 프로세스  
1. 1차 검색(예: 임베딩 기반 벡터 검색)으로 상위 k개 문서 추출.  
2. Reranker 모델에 질문-문서 쌍을 입력.  
3. 각 쌍의 관련성 점수 산출 및 재정렬.  
4. 상위 n개 문서를 LLM의 컨텍스트로 전달하여 답변 생성.

## 장단점

- **장점**  
  - Rerank를 적용하면 단순 벡터 유사도 기반 검색보다 훨씬 정교하게 질문과 관련된 정보를 추출할 수 있다. 
  - 실제로 생성되는 답변의 품질이 크게 향상되며, 도메인 특화 정보나 복잡한 질의에도 높은 정확도를 보인다.

- **단점**  
  - Cross-encoder나 LLM 기반 Rerank는 연산량이 많아 실시간 응답이 필요한 대규모 서비스에선 속도 저하가 발생할 수 있다.
  - 초기 검색 결과(상위 k개)에만 적용하므로, 1차 검색의 품질이 낮으면 Rerank 효과가 제한적일 수 있다.

## CrossEncoder Reranker 예제

In [5]:
from langchain.retrievers import ContextualCompressionRetriever
from langchain.retrievers.document_compressors import CrossEncoderReranker
from langchain_community.cross_encoders import HuggingFaceCrossEncoder

In [6]:
# Cross-encoder 임베딩 모델
reranker_model_id = 'BAAI/bge-reranker-v2-m3'

retriever = get_retriever() # 기본 retriever

reranker = HuggingFaceCrossEncoder(model_name=reranker_model_id)
compressor = CrossEncoderReranker(model=reranker, top_n=5) # 압축방식객체 생성 (cross-encoder 객체)

reranker_retriever = ContextualCompressionRetriever(
    base_retriever=retriever, # 처음 문서를 검색할 retirever
    base_compressor=compressor # base_retriever가 검색한 문서를 rerank하고 압축
)

In [7]:
query = "올림픽과 관련된 논란들을 알려주세요."
base_result = retriever.invoke(query)
rerank_result = reranker_retriever.invoke(query)

In [8]:
base_result_str = [d.page_content for d in base_result]
rerank_result_str = [d.page_content for d in rerank_result]

In [9]:
base_result_str

['올림픽',
 '올림픽',
 '올림픽',
 '1998년에 몇몇 IOC위원들이 2002년 솔트레이크 시티 동계 올림픽 유치 과정에서 미국에게 미국을 올림픽 개최지로 뽑아달라는 뇌물청탁을 받았다는 것이 폭로되었다. 이에 IOC는 사퇴한 IOC위원 4명과 강제 퇴출된 6명에 대한 조사를 했다. 이 스캔들은 이후에 개최지 선정에서 이와 같은 불미스러운 일이 일어나지 않게 하기 위해서 IOC가 개혁에 착수하도록 하는 긍정적인 역할을 하기도 했다.',
 '1998년에 몇몇 IOC위원들이 2002년 솔트레이크 시티 동계 올림픽 유치 과정에서 미국에게 미국을 올림픽 개최지로 뽑아달라는 뇌물청탁을 받았다는 것이 폭로되었다. 이에 IOC는 사퇴한 IOC위원 4명과 강제 퇴출된 6명에 대한 조사를 했다. 이 스캔들은 이후에 개최지 선정에서 이와 같은 불미스러운 일이 일어나지 않게 하기 위해서 IOC가 개혁에 착수하도록 하는 긍정적인 역할을 하기도 했다.',
 '1998년에 몇몇 IOC위원들이 2002년 솔트레이크 시티 동계 올림픽 유치 과정에서 미국에게 미국을 올림픽 개최지로 뽑아달라는 뇌물청탁을 받았다는 것이 폭로되었다. 이에 IOC는 사퇴한 IOC위원 4명과 강제 퇴출된 6명에 대한 조사를 했다. 이 스캔들은 이후에 개최지 선정에서 이와 같은 불미스러운 일이 일어나지 않게 하기 위해서 IOC가 개혁에 착수하도록 하는 긍정적인 역할을 하기도 했다.',
 "BBC 다큐멘터리인 '파노라마'에서는 '매수된 올림픽'이란 주제로 2004년 8월에 방송을 내보내기도 했다. 이때 이 프로그램에서는 2012년 하계 올림픽의 개최지 선정과 관련된 뇌물에 대해서 조사했다. 이 다큐멘터리에서는 특정 후보 도시가 IOC 위원들에게 뇌물수수하는 것이 가능했다고 주장했으며, 특히 파리 시장이었던 베르트랑 들라노에(Bertrand Delanoë)는 영국의 총리인 토니 블레어와 런던올림픽유치위원회가 입후보 규정을 위반했다고 비난했다. 그는 당시 프랑스 대통령이었던 자크 시라크를 목격자로 내세웠지만 시라크

In [10]:
rerank_result_str

["BBC 다큐멘터리인 '파노라마'에서는 '매수된 올림픽'이란 주제로 2004년 8월에 방송을 내보내기도 했다. 이때 이 프로그램에서는 2012년 하계 올림픽의 개최지 선정과 관련된 뇌물에 대해서 조사했다. 이 다큐멘터리에서는 특정 후보 도시가 IOC 위원들에게 뇌물수수하는 것이 가능했다고 주장했으며, 특히 파리 시장이었던 베르트랑 들라노에(Bertrand Delanoë)는 영국의 총리인 토니 블레어와 런던올림픽유치위원회가 입후보 규정을 위반했다고 비난했다. 그는 당시 프랑스 대통령이었던 자크 시라크를 목격자로 내세웠지만 시라크 대통령은 이 분쟁에 휘말려드는 것을 주의했으며 인터뷰를 삼갔다. 결국 베르트랑 들라노에의 주장에 대한 조사는 체계적으로 이루어지지는 않았다. 2006년 동계 올림픽을 유치했던 토리노도 이 논쟁에서 빠져나갈 수 없었다. 이번에는 스위스 국적의 IOC위원 마크 호들러(Marc Hodler)가 이 논쟁의 중심이 되었는데, 이 위원은 스위스 시온의 경쟁 도시였던",
 "BBC 다큐멘터리인 '파노라마'에서는 '매수된 올림픽'이란 주제로 2004년 8월에 방송을 내보내기도 했다. 이때 이 프로그램에서는 2012년 하계 올림픽의 개최지 선정과 관련된 뇌물에 대해서 조사했다. 이 다큐멘터리에서는 특정 후보 도시가 IOC 위원들에게 뇌물수수하는 것이 가능했다고 주장했으며, 특히 파리 시장이었던 베르트랑 들라노에(Bertrand Delanoë)는 영국의 총리인 토니 블레어와 런던올림픽유치위원회가 입후보 규정을 위반했다고 비난했다. 그는 당시 프랑스 대통령이었던 자크 시라크를 목격자로 내세웠지만 시라크 대통령은 이 분쟁에 휘말려드는 것을 주의했으며 인터뷰를 삼갔다. 결국 베르트랑 들라노에의 주장에 대한 조사는 체계적으로 이루어지지는 않았다. 2006년 동계 올림픽을 유치했던 토리노도 이 논쟁에서 빠져나갈 수 없었다. 이번에는 스위스 국적의 IOC위원 마크 호들러(Marc Hodler)가 이 논쟁의 중심이 되었는데, 이 위원은 스위스 시온의 경쟁 도시였던"

# HyDE (Hypothetical Document Embedding)
## 개념
- 질문(query)에 대한 가상의 답변 문서를 생성하고, 이 생성된 가상의 답변 문서를 임베딩하여 검색에 활용하는 기법이다.
- 일반적인 RAG에서 검색은 질문을 임베딩하여 문서 임베딩과 직접 비교한다. 
- 질문("파리는 어떤 도시인가?")과 답변 문서("파리는 프랑스의 수도이며...")는 표현 방식이 다르다는 문제가 있다. 
- 즉 의미적으로는 관련있지만 벡터 공간(임베딩 벡터간의 유사서)에서는 거리가 멀 수가 있다.
- 그래서 HyDE는 질문과 문서가 아니라 질문으로 가상의 답변 문서를 만들고 **가상의 답변문서와 저장된 문서들간의 유사도**를 비교한다.

## HyDE 프로세스

1. 가상 문서 생성
   -  LLM을 사용해 질문에 대한 가상의 답변 문서 생성
   -  이때 성능이 좋은 LLM을 사용하는 것이 좋다.
2. 임베딩 변환
   - `1`에서 생성한 가상 문서를 벡터로 임베딩
3. 유사도 검색
   - 가상 문서 임베딩으로 Vector Store에 저장된 문서들 중 유사한 문서를 검색
4. 답변 생성
   - 검색된 실제 문서를 바탕으로 최종 답변 생성

## 장단점
- **장점**
  - 질문-문서 간 의미적 차이를 해결 해서 정확한 문서 검색 가능
  - 질문 표현 방식에 덜 민감
- **단점**
  - 가상 문서의 품질에 따른 성능 편차가 발생한다.
    - 가상 문서 생성 시 환각(hallucination) 위험
  - 추가적인 LLM 호출로 인한 비용 증가한다.

In [11]:
from langchain_openai import ChatOpenAI
from langchain.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnableLambda
from langchain_core.documents import Document

In [12]:
retriever = get_retriever()
####################
# HyDE Chain을 구성
# - 사용자 질문 -> chain -> 가상의 답변문서
####################
hyde_mode = ChatOpenAI(model='gpt-4.1') # 가상답변을 생성하는 모델은 좋은 모델을 사용해야 한다.
hyde_prompt_template = PromptTemplate(
    template="""
    # Instruction
    다음 질문에 대해서 완전하고 상세한 답변을 실제 사실에 기반해서 작성해 주세요.
    질문과 관련된 내용만으로 답변을 자성합니다.
    답변과 직접적인 연관성이 없는 내용은 답변에 포함시키지 않습니다.

    # 질문:
    {query}
    """
)

hyde_chain = hyde_prompt_template | hyde_mode | StrOutputParser()

In [16]:
query = "동계 올림픽은 언제부터 시작했나요?"
dummy_answer = hyde_chain.invoke({"query": query})

In [19]:
print(dummy_answer)

동계 올림픽은 1924년에 처음 시작되었습니다. 제1회 동계 올림픽은 프랑스의 샤모니(Chamonix)에서 1924년 1월 25일부터 2월 5일까지 개최되었습니다. 이 대회는 당시에는 "국제 동계 스포츠 주간"이라는 명칭으로 열렸으나, 이후 국제올림픽위원회(IOC)에서 공식적으로 제1회 동계 올림픽으로 인정하였습니다.


In [17]:
base_result = retriever.invoke(query)
hyde_result = retriever.invoke(dummy_answer)

In [18]:
[d.page_content for d in base_result], \
[d.page_content for d in hyde_result]

(['동계올림픽\n동계 올림픽은 눈과 얼음을 이용하는 스포츠들을 모아 이루어졌으며 하계 올림픽 때 실행하기 불가능한 종목들로 구성되어 있다. 피겨스케이팅, 아이스하키는 각각 1908년과 1920년에 하계올림픽 종목으로 들어가 있었다. IOC는 다른 동계 스포츠로 구성된 새로운 대회를 만들고 싶어 했고, 로잔에서 열린 1921년 올림픽 의회에서 겨울판 올림픽을 열기로 합의했다. 1회 동계올림픽은 1924년, 프랑스의 샤모니에서 11일간 진행되었고, 16개 종목의 경기가 치러졌다. IOC는 동계 올림픽이 4년 주기로 하계 올림픽과 같은 년도에 열리도록 했다. 이 전통은 프랑스의 알베르빌에서 열린 1992년 올림픽 때까지 지속되었으나, 노르웨이의 릴레함메르에서 열린 1994년 올림픽부터 동계 올림픽은 하계 올림픽이 끝난지 2년후에 개최하였다.',
  '동계올림픽\n동계 올림픽은 눈과 얼음을 이용하는 스포츠들을 모아 이루어졌으며 하계 올림픽 때 실행하기 불가능한 종목들로 구성되어 있다. 피겨스케이팅, 아이스하키는 각각 1908년과 1920년에 하계올림픽 종목으로 들어가 있었다. IOC는 다른 동계 스포츠로 구성된 새로운 대회를 만들고 싶어 했고, 로잔에서 열린 1921년 올림픽 의회에서 겨울판 올림픽을 열기로 합의했다. 1회 동계올림픽은 1924년, 프랑스의 샤모니에서 11일간 진행되었고, 16개 종목의 경기가 치러졌다. IOC는 동계 올림픽이 4년 주기로 하계 올림픽과 같은 년도에 열리도록 했다. 이 전통은 프랑스의 알베르빌에서 열린 1992년 올림픽 때까지 지속되었으나, 노르웨이의 릴레함메르에서 열린 1994년 올림픽부터 동계 올림픽은 하계 올림픽이 끝난지 2년후에 개최하였다.',
  '동계올림픽\n동계 올림픽은 눈과 얼음을 이용하는 스포츠들을 모아 이루어졌으며 하계 올림픽 때 실행하기 불가능한 종목들로 구성되어 있다. 피겨스케이팅, 아이스하키는 각각 1908년과 1920년에 하계올림픽 종목으로 들어가 있었다. IOC는 다른 동계 스포츠로 구성된 새로운 대회를 만

In [22]:
# 최종 chain: hyde_chain | 답변처리 chain
# 답변처리 chain
answer_model = ChatOpenAI(model='gpt-4.1-mini')
template = """
# Instruction:
당신은 정확한 정보 제공을 우선시하는 인공지능 어시스턴트입니다.
주어진 Context에 포함된 정보만 사용해서 질문에 답변하세요.
Context에 질문에 대한 명확한 정보가 있는 경우 그 내용을 바탕으로 답변하세요.
Context에 질문에 대한 명확한 정보없을 경우 "정보가 부족해서 답을 알 수 없습니다." 라고 대답합니다.
절대 Context에 없는 내용을 추측하거나 일반 상식을 이용해 답을 만들어서 대답하지 않습니다.

# Context:
{context}

# 질문:
{query}
"""
answer_prompt_template = PromptTemplate(template=template)

answer_chain = answer_prompt_template | answer_model | StrOutputParser()

##########################################
# 최종 chain = hyde_chain + answer_chain
##########################################

def foramt_docs(docs: list[Document]) -> str:
    return "\n\n".join(doc.page_content for doc in docs)

final_chain = {"context": hyde_chain | retriever | foramt_docs, "query": lambda x: x['query']} | answer_chain

In [25]:
query = "올림픽에서 발생한 논란들에 대해 알려줘."
response = final_chain.invoke({"query": query})

In [26]:
print(response)

올림픽에서 발생한 논란들로는 보이콧, 도핑, 심판 매수, 테러 등이 있습니다. 이러한 문제들은 올림픽이 더욱 굳건히 성장할 수 있는 원동력이 되기도 했습니다.


#  MultiQueryRetriever

## 개념
- 하나의 사용자 질문으로 **여러 개의 다양한 질문을 생성하여 검색**을 수행하는 방법이다.
- 단일 질문의 한계를 극복하고 다각도에서 관련 정보 검색할 수있다.
  - 기본 RAG는 사용자의 질문의 질(quality)에 따라 검색 결과가 좌우된다.
  - 사용자가 한 질문에만 의존하는 것이 아니라 그 질문을 바탕으로 **다양한 의미의 질문들을 생성해서 단일 질문이 가지는 표현의 한계를 보완**한다.
    - 동일한 질문을 다른 각도에서 접근할 수있다.
    - 다양한 어휘와 표현으로 질문을 재구성한다.
- 예)
  - **원본 질문**: "딥러닝의 장점은 무엇인가?"
  - **생성된 질문들**:
    1. "딥러닝이 전통적인 머신러닝보다 나은 점은?"
    2. "딥러닝을 사용하면 얻을 수 있는 이익은?"
    3. "딥러닝의 주요 강점과 특징은?"
    4. "딥러닝 기술의 핵심 우위는?"
## 실행 프로세스

1. 질문 생성   
   - LLM을 사용해 원본 질문을 3-5개의 서로 다른 질문으로 변환
2. 병렬 검색
   - 생성된 각 질문으로 독립적으로 문서 검색 수행
3. 결과 통합
   - 여러 검색 결과를 하나로 병합
4. 중복 제거
   - 동일한 문서가 여러 번 검색된 경우 중복 제거
5. 최종 답변
   - 통합된 문서 세트를 바탕으로 답변 생성

## 장단점
- **장점**
    - 단일 질문으로 놓칠 수 있는 관련 문서 발견 수 있다.
    - 사용자 질문 표현 방식의 한계 극복
    - 더 포괄적이고 완전한 정보 검색 및 수집을 할 수있다.
- **단점**
    - 여러 번의 LLM 호출과 검색 수행이 실행 되므로 **계산비용, 토큰비용, 응답시간이 증가한다.**
    - 생성된 질문의 품질에 따른 성능 편차가 있을 수 있다.
    - 생성된 질문에 때라 원래 질문과 관련성 낮은 문서도 검색될 수 있어 최종 답변을 방해하는 노이즈가 증가할 수있다.

In [27]:
from langchain.retrievers.multi_query import MultiQueryRetriever

model = ChatOpenAI(model='gpt-4.1')
retriever = get_retriever()

multi_query_retriever = MultiQueryRetriever.from_llm(
    retriever=retriever,
    llm=model
)
# 1. 입력받은 질문을 이용해서 가상의 질문 3개 생성
# 2. 생성된 질문을 retriever에 전달해서 문서들을 검색
# 3. 중복된 문서들을 제거하고 문서 반환

In [28]:
result_docs = multi_query_retriever.invoke("올림픽 논란은?")

In [29]:
print(foramt_docs(result_docs))

현대 올림픽에서는 프로 선수의 참가 불허가 많은 분쟁을 가져왔다. 1912년 하계 올림픽의 근대 5종 경기와 10종 경기에서 우승한 짐 소프는 올림픽에 나가기 전에 준프로야구선수로 활동했다는 게 나중에 밝혀져 메달이 박탈되었다. 소프는 후에 동정적 여론의 힘을 업고 1983년에 메달을 돌려받게 된다. 1936년 동계 올림픽 때 스위스와 오스트리아 스키선수들은 돈을 벌기 위해 스포츠를 했는데 이러한 행동이 아마추어 정신에 위배된다고 결정되어 그들은 스키종목에 참가할 수 없었다.

현대 올림픽에서는 프로 선수의 참가 불허가 많은 분쟁을 가져왔다. 1912년 하계 올림픽의 근대 5종 경기와 10종 경기에서 우승한 짐 소프는 올림픽에 나가기 전에 준프로야구선수로 활동했다는 게 나중에 밝혀져 메달이 박탈되었다. 소프는 후에 동정적 여론의 힘을 업고 1983년에 메달을 돌려받게 된다. 1936년 동계 올림픽 때 스위스와 오스트리아 스키선수들은 돈을 벌기 위해 스포츠를 했는데 이러한 행동이 아마추어 정신에 위배된다고 결정되어 그들은 스키종목에 참가할 수 없었다.

현대 올림픽에서는 프로 선수의 참가 불허가 많은 분쟁을 가져왔다. 1912년 하계 올림픽의 근대 5종 경기와 10종 경기에서 우승한 짐 소프는 올림픽에 나가기 전에 준프로야구선수로 활동했다는 게 나중에 밝혀져 메달이 박탈되었다. 소프는 후에 동정적 여론의 힘을 업고 1983년에 메달을 돌려받게 된다. 1936년 동계 올림픽 때 스위스와 오스트리아 스키선수들은 돈을 벌기 위해 스포츠를 했는데 이러한 행동이 아마추어 정신에 위배된다고 결정되어 그들은 스키종목에 참가할 수 없었다.

올림픽에서 약물 복용 양성 반응이 나와서 메달을 박탈당한 첫 번째 사례로는 1968년 하계 올림픽의 근대 5종 경기에 출전해 동메달을 딴 한스 군나르 리렌바르가 있다. 그는 경기 후 도핑검사 결과 알코올을 복용한 것으로 확인되어 메달을 박탈당했다. 도핑 양성 반응으로 메달을 박탈당한 것으로 가장 유명한 사람은 1988년 하계 올림픽 육상 

# MapReduce RAG 방식

## 개요

- RAG(Retrieval-Augmented Generation)에서 검색된 문서들 중 질문과 관련성이 높은 문서만을 선별하여 더 정확한 답변을 생성하는 방법이다.
- 검색된 문서들 중에서 질문 답변에 실제로 도움이 되는 문서만을 LLM을 통해 선별한 후 전달하는 방식이다.

## MapReduce 방식 프로세스
1. Map (문서 검색)
  - 벡터스토어에서 질문과 유사한 문서들을 의미적 유사도 검색으로 찾는다.
  - 이 단계에서는 단순 벡터 유사도만 고려하므로 질문과 직접적인 관련이 없는 문서도 포함될 수 있다.
2. Reduce (문서 선별 및 요약)
   - 검색된 각 문서가 질문 답변에 실제로 도움이 되는지 LLM에게 평가 요청한다.
   - 관련성이 높은 문서들만 선별하여 요약하거나 결합한다.
   - 필요시 여러 문서의 정보를 통합하여 더 응답에 적합한 컨텍스트를 생성한다.
3.  Generate (최종 답변 생성)
    - 질문과 선별된 컨텍스트를 함께 LLM에 전달하여 최종 답변을 생성한다.

## 장단점

- **장점**
  - **높은 정확도**: 질문과 직접 관련된 정보만 사용하여 더 정확한 답변을 생성한다.
  - **노이즈 제거**: 유사하지만 관련 없는 정보로 인한 혼동을 방지한다.
  - **컨텍스트 최적화**: 제한된 토큰 범위 내에서 가장 유용한 정보만 전달한다.
  - **확장성**: 많은 문서가 검색되어도 중요한 정보만 선별하여 처리할 수 있다.
- 단점
  - **추가 비용**: 문서 선별을 위한 LLM 호출로 인한 비용이 증가한다.
  - **처리 시간**: 문서 평가 단계가 추가되어 응답 속도가 저하된다.
  - **복잡성**: 구현과 관리가 더 복잡하다.
  - **의존성**: 문서 선별 성능이 LLM의 판단 능력에 크게 의존한다.

In [30]:
from langchain_openai import ChatOpenAI
from langchain.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from dotenv import load_dotenv

load_dotenv()

True

In [31]:
retriever = get_retriever()

In [32]:
# 질문과 문서간의 관련성을 확인하는 chain
reducer_prompt_template = PromptTemplate(
    template="""
    #Instruction:
    Context의 내용이 질문과 관련성이 있으면 입력된 context를 반환하고 관련성이 없으면 아무것도 출력하지 마세요.
    - 관련성 판단기준
        - Context에 질문에 대한 직접적인 답변이나 답을 위한 단서가 있어야 한다.
        - Context의 정보가 질문을 해결하는데 도움을 줘야한다.

    # Context:
    {context}

    # 질문:
    {query}

    # output indicator:
    - 질문과 context가 관련있으면 context 내용만 정확히 그대로 반환한다. 추가 설명, 내용 수정, 요약 등 변경을 금지한다.
    - 질문과 context가 관련성이 없으면 아무것도 출력하지 않습니다.
"""
)
reducer_model = ChatOpenAI(model='gpt-4.1-mini')
reducer_chain = reducer_prompt_template | reducer_model | StrOutputParser()

In [33]:
# 테스트
reducer_chain.invoke({"query": "사과에 좋은점을 알려줘.", "context": "올림픽은 IOC가 관리합니다."})

''

In [34]:
# 테스트
reducer_chain.invoke({"query": "사과에 좋은점을 알려줘.", "context": "좋은 영양소가 많습니다."})

'좋은 영양소가 많습니다.'

In [ ]:
###################
# retriever로 문서 조회 -> reducer_chain에서 질문과의 관련성 있는 문서만 추리기.

@chain
def map_doc(input: dict) -> str:
    """
    Retriever가 조회한 문서들과 query를 받아서 reducer_chain을 이용해 질문과 각 문서간의 
    """